In [2]:
import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [7]:
import optuna
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

def objective(trial):
    # Load a different dataset
    (data, target) = sklearn.datasets.load_iris(return_X_y=True)
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    param = {
        "verbosity": 0,
        "objective": "multi:softmax",  # Multi-class classification objective
        "num_class": 3,  # Number of classes in the Iris dataset

        "tree_method": "exact",
        
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
      
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
       
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")
     


[I 2023-05-27 03:46:48,759] A new study created in memory with name: no-name-00866a7f-0c93-4625-9c43-773716372df3
[I 2023-05-27 03:46:48,775] Trial 0 finished with value: 0.8157894736842105 and parameters: {'booster': 'gblinear', 'lambda': 2.12555925896576e-07, 'alpha': 4.1653517782189e-08, 'subsample': 0.31282639824615627, 'colsample_bytree': 0.6915264521057713}. Best is trial 0 with value: 0.8157894736842105.
[I 2023-05-27 03:46:48,797] Trial 1 finished with value: 0.9473684210526315 and parameters: {'booster': 'dart', 'lambda': 1.5621424000139158e-06, 'alpha': 3.411726647160833e-08, 'subsample': 0.9251601708206729, 'colsample_bytree': 0.43475249439924657, 'max_depth': 5, 'min_child_weight': 9, 'eta': 1.7980785075569594e-05, 'gamma': 4.8390919218788045e-08, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 2.4378256086408806e-07, 'skip_drop': 0.0009057236817175958}. Best is trial 1 with value: 0.9473684210526315.
[I 2023-05-27 03:46:48,820] 

Number of finished trials:  100
Best trial:
  Value: 1.0
  Params: 
    booster: gbtree
    lambda: 2.5105399586296754e-07
    alpha: 8.074238783134525e-06
    subsample: 0.5144772459736243
    colsample_bytree: 0.7561582151554103
    max_depth: 3
    min_child_weight: 4
    eta: 0.0012116741408408481
    gamma: 1.020825120360011e-08
    grow_policy: lossguide


##### Differences between Optuna, Grid Search and Random Search

Optuna is a more advanced and efficient approach to hyperparameter optimization compared to Grid Search and Random Search. It intelligently explores the search space and adapts the search based on previous trials, resulting in faster convergence and better performance. However, Grid Search and Random Search are simpler to implement and understand, and they can still be useful in scenarios where the search space is small or a quick exploration is needed.